In [1]:
import cv2
import os
import numpy as np

# Set the directory where your images and labels are stored
image_dir = "/Users/deepaklenka/Downloads/dataset/images"
label_dir = "/Users/deepaklenka/Downloads/dataset/labels"

# Create lists to store the images, class labels, and bounding box coordinates
images = []
class_labels = []
bbox_coordinates = []

# Loop through the images
for i in range(1, 81):  # Assuming you have 80 images
    image_filename = f"{i}.jpg"
    label_filename = f"{i}.txt"

    # Construct the full path to the image
    image_path = os.path.join(image_dir, image_filename)

    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    # Load and preprocess the image
    img = cv2.imread(image_path)  # Load the image using OpenCV

    # Check if the image is empty (invalid)
    if img is None:
        print(f"Error loading image: {image_path}")
        continue

    img = cv2.resize(img, (224, 224))  # Resize the image
    img = img / 255.0  # Normalize pixel values

    images.append(img)

    # Load and process the label file
    label_path = os.path.join(label_dir, label_filename)
    with open(label_path, 'r') as label_file:
        label_data = label_file.read().strip().split(' ')  # Split by space
        class_label = label_data[0]  # First element is the class label
        bbox_values = [float(val) for val in label_data[1:]]  # The rest are bounding box coordinates
        class_labels.append(class_label)
        bbox_coordinates.append(bbox_values)

# Convert the lists to NumPy arrays
images = np.array(images)
class_labels = np.array(class_labels)

# You can't directly convert bbox_coordinates to a NumPy array due to varying lengths.
# If all images have the same number of ROIs, you can convert it to a NumPy array.
# If not, you may need to keep it as a list of lists for further processing.

# Now, you have your preprocessed images in 'images', class labels in 'class_labels'.
# Bbox_coordinates is a list of lists representing bounding box coordinates.

In [2]:
!pip install roboflow

  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/4f/2c/13ef4a1811365208f7441dae564ea57dad4b58762196b7d8e65860b0cc07/roboflow-1.1.7-py3-none-any.whl.metadata
  Using cached roboflow-1.1.7-py3-none-any.whl.metadata (9.0 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Obtaining dependency information for opencv-python-headless==4.8.0.74 from https://files.pythonhosted.org/packages/9d/df/88bc1f767d880d4629d71d6d3907bf89aea4506b828e9ede7d3846a0912d/opencv_python_headless-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl.metadata
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Obtaining dependency information for supervision from https://files.pythonhosted.org/packages/c0/59/6bc863f530c0ba5da4c565fd40f17b2b333c2ab28b78320

In [ ]:
import requests
import json

# Replace with your API key and project ID
api_key = "oVGzpi8kyiMhWtwwA208"
project_id = "stock-chart"

# URL for making predictions
url = f"https://detect.roboflow.com/stock-chart/2?api_key=oVGzpi8kyiMhWtwwA208"

# Path to your image file
image_path = "/Users/deepaklenka/Downloads/deepu.jpg"  # Replace with the path to your image

# Confidence and overlap thresholds
confidence = 40
overlap = 30

headers = {
    "Authorization": f"Bearer {api_key}"
}

files = {"file": (image_path, open(image_path, "rb"))}

# Send the API request
response = requests.post(url, headers=headers, files=files, data={"confidence": confidence, "overlap": overlap})

# Print the entire API response
print(response.text)

# Check if the response is successful and contains JSON data
if response.status_code == 200:
    try:
        predictions = json.loads(response.text)
        print(predictions)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
else:
    print(f"Request failed with status code: {response.status_code}")

In [ ]:
import cv2

# Load the image
image = cv2.imread(image_path)

# Loop through the predictions
for prediction in predictions['predictions']:
    class_name = prediction['class']
    x, y, width, height = int(prediction['x']), int(prediction['y']), int(prediction['width']), int(prediction['height'])

    # Draw a bounding box
    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)

    # Display the class name and confidence
    label = f"{class_name} ({prediction['confidence']:.2f})"
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow("Predictions", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import requests
import cv2
import numpy as np

# Replace with your API key and project ID
api_key = "oVGzpi8kyiMhWtwwA208"
project_id = "stock-chart"

# URL for making predictions
url = f"https://detect.roboflow.com/stock-chart/2?api_key=oVGzpi8kyiMhWtwwA208"


# URL of the image hosted elsewhere
image_url = "https://dwptxtcjzzofa.cloudfront.net/webp/fm48v81/st_best-stock-charts_sc_acp_o.webp"  # Replace with the URL of the image

# Confidence and overlap thresholds
confidence = 40
overlap = 30

headers = {
    "Authorization": f"Bearer {api_key}"
}

# Fetch the image from the URL
response = requests.get(image_url)
image_bytes = response.content

# Create an image object from the byte data
image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)

# Encode the image as a byte array
_, image_encoded = cv2.imencode('.jpg', image)
image_bytes = image_encoded.tobytes()

files = {"file": ("image.jpg", image_bytes)}

# Send the API request
response = requests.post(url, headers=headers, files=files, data={"confidence": confidence, "overlap": overlap})

# Print the entire API response
print(response.text)

# Check if the response is successful and contains JSON data
if response.status_code == 200:
    try:
        predictions = json.loads(response.text)
        print(predictions)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
else:
    print(f"Request failed with status code: {response.status_code}")